# Testing a Custom Redis Server with Raw Sockets

This notebook provides an interactive way to test a custom, Redis-compatible server by sending **raw RESP (REdis Serialization Protocol) messages** over a TCP socket. This is similar to using a tool like `netcat`.

This method is excellent for low-level testing of your server's protocol parser.

### Instructions:
1. Make sure your custom Zig-based Redis server is running.
2. Run the cells sequentially by pressing `Shift + Enter`.
3. Observe the output of each cell. The `repr()` output shows the exact bytes received from the server, including `\r\n`.

In [1]:
# Step 1: Import necessary libraries
import socket
from typing import List, Union

In [2]:
# Step 2: Configure the connection to your server
REDIS_HOST = "localhost"
REDIS_PORT = 8080
CONN_TIMEOUT = 2.0

In [3]:
# Step 3: Define the helper function to send RESP commands

def send_resp_command(command: List[Union[str, int]]):
    """
    Constructs a RESP message, sends it via a raw socket, 
    and prints the command and response.
    """
    try:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.settimeout(CONN_TIMEOUT)
            s.connect((REDIS_HOST, REDIS_PORT))

            # Build the RESP message as an array of bulk strings
            resp_parts = [f"*{len(command)}"]
            for part in command:
                part_str = str(part)
                resp_parts.append(f"${len(part_str)}")
                resp_parts.append(part_str)

            resp_message = "\r\n".join(resp_parts) + "\r\n"
            encoded_message = resp_message.encode('utf-8')
            
            print(f"--- Sending Command: {' '.join(map(str, command))} ---")
            print(f"Raw RESP Sent: {repr(encoded_message)}")

            s.sendall(encoded_message)
            response_bytes = s.recv(4096)
            
            print(f"Raw RESP Received: {repr(response_bytes)}")
            print(f"Decoded Response: '{response_bytes.decode('utf-8').strip()}'\n")
            
    except (socket.timeout, ConnectionRefusedError) as e:
        print(f"ERROR: Connection to {REDIS_HOST}:{REDIS_PORT} failed. Is the server running?")
        print(f"Details: {e}\n")

--- 
## Command Tests
Now we will run tests for each command.

In [4]:
# Before starting, let's clear the database to ensure a clean state.
# Expected response is a Simple String: +OK
send_resp_command(["FLUSHDB"])

--- Sending Command: FLUSHDB ---
Raw RESP Sent: b'*1\r\n$7\r\nFLUSHDB\r\n'
Raw RESP Received: b'+OK\r\n'
Decoded Response: '+OK'



In [5]:
# Test: PING
# Expected response is a Simple String: +PONG
send_resp_command(["PING"])

--- Sending Command: PING ---
Raw RESP Sent: b'*1\r\n$4\r\nPING\r\n'
Raw RESP Received: b'$4\r\nPONG\r\n'
Decoded Response: '$4
PONG'



In [6]:
# Test: ECHO
# Expected response is a Bulk String: $11\r\nHello RESP\r\n
send_resp_command(["ECHO", "Hello RESP"])

--- Sending Command: ECHO Hello RESP ---
Raw RESP Sent: b'*2\r\n$4\r\nECHO\r\n$10\r\nHello RESP\r\n'
Raw RESP Received: b'$10\r\nHello RESP\r\n'
Decoded Response: '$10
Hello RESP'



In [7]:
# Test: SET and GET
# SET should respond with a Simple String: +OK
# GET should respond with a Bulk String: $8\r\nRawValue\r\n
send_resp_command(["SET", "raw_key", "RawValue"])
send_resp_command(["GET", "raw_key"])

--- Sending Command: SET raw_key RawValue ---
Raw RESP Sent: b'*3\r\n$3\r\nSET\r\n$7\r\nraw_key\r\n$8\r\nRawValue\r\n'
Raw RESP Received: b'+OK\r\n'
Decoded Response: '+OK'

--- Sending Command: GET raw_key ---
Raw RESP Sent: b'*2\r\n$3\r\nGET\r\n$7\r\nraw_key\r\n'
Raw RESP Received: b'$8\r\nRawValue\r\n'
Decoded Response: '$8
RawValue'



In [8]:
# Test: GET non-existent key
# Expected response is a Null Bulk String: $-1
send_resp_command(["GET", "no_such_key"])

--- Sending Command: GET no_such_key ---
Raw RESP Sent: b'*2\r\n$3\r\nGET\r\n$11\r\nno_such_key\r\n'
Raw RESP Received: b'$-1\r\n'
Decoded Response: '$-1'



In [9]:
# Test: EXISTS and DEL
# Both commands respond with an Integer reply, e.g., :1 or :0
send_resp_command(["SET", "another_key", "xyz"])
send_resp_command(["EXISTS", "another_key"])
send_resp_command(["DEL", "another_key"])
send_resp_command(["EXISTS", "another_key"])

--- Sending Command: SET another_key xyz ---
Raw RESP Sent: b'*3\r\n$3\r\nSET\r\n$11\r\nanother_key\r\n$3\r\nxyz\r\n'
Raw RESP Received: b'+OK\r\n'
Decoded Response: '+OK'

--- Sending Command: EXISTS another_key ---
Raw RESP Sent: b'*2\r\n$6\r\nEXISTS\r\n$11\r\nanother_key\r\n'
Raw RESP Received: b':1\r\n'
Decoded Response: ':1'

--- Sending Command: DEL another_key ---
Raw RESP Sent: b'*2\r\n$3\r\nDEL\r\n$11\r\nanother_key\r\n'
Raw RESP Received: b':1\r\n'
Decoded Response: ':1'

--- Sending Command: EXISTS another_key ---
Raw RESP Sent: b'*2\r\n$6\r\nEXISTS\r\n$11\r\nanother_key\r\n'
Raw RESP Received: b':0\r\n'
Decoded Response: ':0'



In [10]:
# Test: INCR, DECR, INCRBY, DECRBY
# These commands respond with an Integer reply.
send_resp_command(["SET", "num_key", 10])
send_resp_command(["INCR", "num_key"])
send_resp_command(["DECR", "num_key"])
send_resp_command(["INCRBY", "num_key", 5])
send_resp_command(["DECRBY", "num_key", 3])

--- Sending Command: SET num_key 10 ---
Raw RESP Sent: b'*3\r\n$3\r\nSET\r\n$7\r\nnum_key\r\n$2\r\n10\r\n'
Raw RESP Received: b'+OK\r\n'
Decoded Response: '+OK'

--- Sending Command: INCR num_key ---
Raw RESP Sent: b'*2\r\n$4\r\nINCR\r\n$7\r\nnum_key\r\n'
Raw RESP Received: b':11\r\n'
Decoded Response: ':11'

--- Sending Command: DECR num_key ---
Raw RESP Sent: b'*2\r\n$4\r\nDECR\r\n$7\r\nnum_key\r\n'
Raw RESP Received: b':10\r\n'
Decoded Response: ':10'

--- Sending Command: INCRBY num_key 5 ---
Raw RESP Sent: b'*3\r\n$6\r\nINCRBY\r\n$7\r\nnum_key\r\n$1\r\n5\r\n'
Raw RESP Received: b':15\r\n'
Decoded Response: ':15'

--- Sending Command: DECRBY num_key 3 ---
Raw RESP Sent: b'*3\r\n$6\r\nDECRBY\r\n$7\r\nnum_key\r\n$1\r\n3\r\n'
Raw RESP Received: b':12\r\n'
Decoded Response: ':12'



In [11]:
# Test: HSET, HGET, HGETALL
# HSET returns an Integer (fields added).
# HGET returns a Bulk String.
# HGETALL returns an Array of Bulk Strings.
send_resp_command(["HSET", "raw_hash", "name", "Bob"])
send_resp_command(["HSET", "raw_hash", "email", "bob@example.com"])
send_resp_command(["HGET", "raw_hash", "name"])
send_resp_command(["HGETALL", "raw_hash"])

--- Sending Command: HSET raw_hash name Bob ---
Raw RESP Sent: b'*4\r\n$4\r\nHSET\r\n$8\r\nraw_hash\r\n$4\r\nname\r\n$3\r\nBob\r\n'
Raw RESP Received: b':1\r\n'
Decoded Response: ':1'

--- Sending Command: HSET raw_hash email bob@example.com ---
Raw RESP Sent: b'*4\r\n$4\r\nHSET\r\n$8\r\nraw_hash\r\n$5\r\nemail\r\n$15\r\nbob@example.com\r\n'
Raw RESP Received: b':1\r\n'
Decoded Response: ':1'

--- Sending Command: HGET raw_hash name ---
Raw RESP Sent: b'*3\r\n$4\r\nHGET\r\n$8\r\nraw_hash\r\n$4\r\nname\r\n'
Raw RESP Received: b'$3\r\nBob\r\n'
Decoded Response: '$3
Bob'

--- Sending Command: HGETALL raw_hash ---
Raw RESP Sent: b'*2\r\n$7\r\nHGETALL\r\n$8\r\nraw_hash\r\n'
Raw RESP Received: b'*4\r\n$4\r\nname\r\n$3\r\nBob\r\n$5\r\nemail\r\n$15\r\nbob@example.com\r\n'
Decoded Response: '*4
$4
name
$3
Bob
$5
email
$15
bob@example.com'



In [12]:
# Test: TYPE
# Responds with a Simple String: +string, +hash, or +none
send_resp_command(["SET", "str_type_key", "v"])
send_resp_command(["HSET", "hash_type_key", "f", "v"])
send_resp_command(["TYPE", "str_type_key"])
send_resp_command(["TYPE", "hash_type_key"])
send_resp_command(["TYPE", "none_type_key"])

--- Sending Command: SET str_type_key v ---
Raw RESP Sent: b'*3\r\n$3\r\nSET\r\n$12\r\nstr_type_key\r\n$1\r\nv\r\n'
Raw RESP Received: b'+OK\r\n'
Decoded Response: '+OK'

--- Sending Command: HSET hash_type_key f v ---
Raw RESP Sent: b'*4\r\n$4\r\nHSET\r\n$13\r\nhash_type_key\r\n$1\r\nf\r\n$1\r\nv\r\n'
Raw RESP Received: b':1\r\n'
Decoded Response: ':1'

--- Sending Command: TYPE str_type_key ---
Raw RESP Sent: b'*2\r\n$4\r\nTYPE\r\n$12\r\nstr_type_key\r\n'
Raw RESP Received: b'+string\r\n'
Decoded Response: '+string'

--- Sending Command: TYPE hash_type_key ---
Raw RESP Sent: b'*2\r\n$4\r\nTYPE\r\n$13\r\nhash_type_key\r\n'
Raw RESP Received: b'+hash\r\n'
Decoded Response: '+hash'

--- Sending Command: TYPE none_type_key ---
Raw RESP Sent: b'*2\r\n$4\r\nTYPE\r\n$13\r\nnone_type_key\r\n'
Raw RESP Received: b'+none\r\n'
Decoded Response: '+none'



In [13]:
# Test: GETDEL
# Responds with the Bulk String value of the key, or Null Bulk String.
send_resp_command(["SET", "getdel_raw_key", "this will disappear"])
send_resp_command(["GETDEL", "getdel_raw_key"])
send_resp_command(["EXISTS", "getdel_raw_key"])

--- Sending Command: SET getdel_raw_key this will disappear ---
Raw RESP Sent: b'*3\r\n$3\r\nSET\r\n$14\r\ngetdel_raw_key\r\n$19\r\nthis will disappear\r\n'
Raw RESP Received: b'+OK\r\n'
Decoded Response: '+OK'

--- Sending Command: GETDEL getdel_raw_key ---
Raw RESP Sent: b'*2\r\n$6\r\nGETDEL\r\n$14\r\ngetdel_raw_key\r\n'
Raw RESP Received: b'$19\r\nthis will disappear\r\n'
Decoded Response: '$19
this will disappear'

--- Sending Command: EXISTS getdel_raw_key ---
Raw RESP Sent: b'*2\r\n$6\r\nEXISTS\r\n$14\r\ngetdel_raw_key\r\n'
Raw RESP Received: b':0\r\n'
Decoded Response: ':0'

